# 共享变量:累加器和广播变量


In [2]:
import pyspark

from pyspark import SparkConf,SparkContext

In [3]:
conf = SparkConf().setMaster("local").setAppName("My APP")
sc = SparkContext(conf = conf)

# 1.读取文本文件
lines = sc.textFile('file:///Users/fire/jupyter/data/ball2018.txt')
# 存储文本
#lines.saveAsTextFile('file:///Users/fire/jupyter/data/ball20182.txt')

### 累加器
用法：
1. 通过在驱动器中调用SparkContext.accumulator(initialValue)方法，创建出存有初始值的累加器。返回值为org.apache.spark.Accumulator[T]对象，其中T是初始值initialValue的类型。
2. Spark闭包里的执行器代码可以使用累加器的+=方法增加累加器的值。
3. 驱动器程序可以调用累加器的value属性来访问累加器的值。

In [27]:
# 累加空行
# 创建Accumulator[Int] 并初始化为0
blankLines = sc.accumulator(0)

def extractCallSigns(line):
    global blankLines # 访问全局变量
    if (line == ""):
        blankLines += 1
    return line.split(" ")

callSigns = lines.flatMap(extractCallSigns)
callSigns.saveAsTextFile('file:///Users/fire/jupyter/data/' + "/callsigns")
print("Blank lines: %d" % blankLines.value)

Blank lines: 0


### 广播变量 
广播变量可以让程序高效地向所有工作节点发送一个较大的只读值，以供一个或多个Spark操作使用。

用法：
1. 通过类型T的对象调用SparkContext.broadcast创建出一个Broadcast[T]对象。任何可以序列话的类型都可以这么实现。
2. 通过value属性访问该对象的值。
3. 变量只会被发到各个节点一次，应作为只读值处理（修改这个值不会影响到其他节点）。

In [29]:
#使用广播变量查询国家
# 查询RDD contactCounts中的呼号的对应位置。
# 将呼号前缀读取为国家代码来进行查询
signPrefixes = sc.broadcast(loadCallSignTable())

def processSignCount(sign_count, signPrefixes):
    country = lookupCountry(sign_count[0], signPrefixes.value)
    count = sign_count[1]
    return (country, count)

countryContactCounts = (contactCounts.map(processSignCount).reduceByKey((lambda x,y:x+y)))
countryContactCounts.saveAsTextFile(outpitDir+"/countries.txt")

NameError: name 'loadCallSignTable' is not defined

### 基于分区进行操作
避免为每个元素进行重复的配置工作

In [36]:
# mapPartitions() 提供该分区中元素的迭代器，返回元素的迭代器
# 不使用 mapPartitions 求平均值
def combineCtrs(c1,c2):
    return (c1[0]+c2[0],c1[1]+c2[1])

def basicAvg(nums):
    nums.map(lambda num:(num,1)).reduce(combineCtrs)

nums = sc.parallelize([[1,2,3],[3,2,0]])
#print(basicAvg(nums))  #Py4JJavaError
# 使用 mapPartitions 求平均值
def partitonCtr(nums):
    #计算分区的sumCounter
    sumCount = [0,0]
    for num in nums:
        sumCount[0] += num
        sumCount[1] += 1
    return [sumCount]

def fastAvg(nums):
    #计算平均值
    sumCount = nums.mapPartitions(partitionCtr).reduce(combineCtrs)
    return sumCount[0] / float(sumCount[1])

### 数值RDD操作
即RDD.count() .mean() .sum() .max() 等等。

In [38]:
# 移除异常值
# 需要吧string类型RDD转为数字类型
distanceNumerics = distances.map(lambda string:float(string))
stats = distanceNumerics.stats()
stddev = stdts.stdev()
mean = stats.mean()
reasonableDistances = distanceNumerics.filter(lambda x: math.fabs(x-mean)<3 *stddev)
print(reasonableDistances.collect())

NameError: name 'distances' is not defined